# Analyzing features from pre-trained models for anomaly detection 

In [ ]:
# To autoreload external functions
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
from typing import Optional
import os
from pathlib import Path
from PIL import Image
import math
import random
import numpy as np
import cv2
from sklearn.decomposition import PCA
from sklearn.covariance import LedoitWolf
from sklearn.metrics import roc_curve, auc
import umap

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.models import list_models, get_model
from torchvision.models.feature_extraction import (
    get_graph_node_names,
    create_feature_extractor,
)

import matplotlib.pyplot as plt
import seaborn as sns

import rootutils

root = rootutils.setup_root(Path.cwd(), dotenv=True, pythonpath=True, cwd=False)

from src.visualization.utils import (
    save_plot_from_notbook_for_jekyll,
    bokeh_notebook_setup,
)
from src.visualization.image import (
    plot_img_rgba,
    add_seg_on_img,
    add_score_map_on_img,
)
from src.visualization.features import plot_feature_samples, plot_feature_3d_samples, plot_labelled_feature_samples, plot_labelled_feature_3d_samples

## Setup

In [ ]:
%matplotlib ipympl

In [ ]:
bokeh_notebook_setup()

# make random number generator repeatable
seed = 1
random.seed(seed)
np.random.seed(seed)

sns.set_style('darkgrid')

In [ ]:
data_path = Path("../data/raw/wood")
output_path = Path("./logs")

## Introduction

- take feature extraction approach as in previous blog post
- apply normal PCA to reduce to 2 dimensions
- repeat with modified PCA to keep the feature combinations with smallest variance
- plot and compar normal and anomalous features
- repeat experiment with PCA reduction to 3 dimensions
  - explore 3d plots with bokeh

## Dataset

Use again 'Metal Nut' category from [MVTec anomaly detection dataset](https://www.mvtec.com/company/research/datasets/mvtec-ad)

## Feature Extraction

See last post

In [ ]:
# list_models()

In [ ]:
class Config:
    model_name = "convnext_base"
    layer_names = ["features.6", "features.7"]
    img_shape = (224, 224)  # height, width
    batch_size = 16
    num_workers: int = 8  # adjust to the number of processing cores you want to use
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    n_feats = None # number of features (depends on the chosen layer)

In [ ]:
class GlobalFeatureExtractor(nn.Module):
    def __init__(self, feature_extractor):
        super().__init__()

        self.feature_extractor = feature_extractor
        self.pool_layer = torch.nn.AdaptiveAvgPool2d(output_size=1)

    def forward(self, x):
        feature_dict = self.feature_extractor(x)
        for k, v in feature_dict.items():
            feature_dict[k] = self.pool_layer(v)

        return feature_dict

In [ ]:
backbone = get_model(Config.model_name, weights="DEFAULT")

In [ ]:
# train_nodes, eval_nodes = get_graph_node_names(backbone)
# train_nodes

In [ ]:
feature_extractor = create_feature_extractor(backbone, return_nodes=Config.layer_names)
feature_extractor=GlobalFeatureExtractor(feature_extractor)

for param in feature_extractor.parameters():
    param.requires_grad = False

In [ ]:
def get_features(imgs, extractor, cfg):
    imgs = imgs.to(cfg.device)

    with torch.no_grad():
        feature_dict = extractor(imgs)

    layers = list(feature_dict.keys())
    l_feats = [feature_dict[layer].squeeze((2, 3)) for layer in layers]
    feats = torch.cat(l_feats, 1)
    feats = feats.cpu().numpy()

    return feats


def get_ground_truths(labels, cfg):
    labels = labels.numpy()
    labels = labels.reshape(-1)

    return labels

### Training Data

In [ ]:
class TrainDataset(Dataset):
    def __init__(
        self,
        data_path: os.PathLike,
        transforms: Optional[A.Compose] = None,
        N_train: Optional[int] = None,
    ):
        super(TrainDataset).__init__()

        self.img_paths = list(data_path.iterdir())
        self.transforms = transforms

        if N_train is not None and len(self.img_paths) > N_train:
            self.img_paths = random.sample(self.img_paths, N_train)

    def __getitem__(self, index: int):
        img_path = self.img_paths[index]

        img = Image.open(img_path)
        img = img.convert("RGB")
        img = np.array(img)

        if self.transforms:
            img = self.transforms(image=img)["image"]

        return img

    def __len__(self) -> int:
        return len(self.img_paths)

In [ ]:
train_path = data_path / "train/good"
val_path = data_path / "test"
gt_path = data_path / "ground_truth"

default_transforms = A.Compose(
    [
        A.Resize(Config.img_shape[0], Config.img_shape[1]),
        A.Normalize(
            mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0
        ),
        ToTensorV2(),
    ]
)

train_ds = TrainDataset(train_path, transforms=default_transforms)

train_dl = DataLoader(
    train_ds,
    batch_size=Config.batch_size,
    shuffle=False,
    num_workers=Config.num_workers,
)

In [ ]:
imgs = next(iter(train_dl))
feats_shapes = []

for layer_name in Config.layer_names:
    feats_shapes.append(feature_extractor(imgs)[layer_name].shape)

Config.n_feats = sum([fs[1] for fs in feats_shapes])
print("n feats:", Config.n_feats)

In [ ]:
train_features = np.empty((len(train_ds), Config.n_feats), dtype=np.float32)
feature_extractor = feature_extractor.to(Config.device)
i_mem = 0

for i, imgs in enumerate(train_dl):
    feats = get_features(imgs, feature_extractor, Config)
    train_features[i_mem : i_mem + feats.shape[0]] = feats
    i_mem += feats.shape[0]

In [ ]:
print("Train features shape:", train_features.shape)

### Validation Data

In [ ]:
class ValidationDataset(Dataset):
    def __init__(
        self,
        data_path: os.PathLike,
        gt_path: os.PathLike,
        transforms: Optional[A.Compose] = None,
    ):
        super(ValidationDataset).__init__()

        self.img_paths = list()
        self.gt_paths = list()

        gt_class_paths = list(data_path.iterdir())
        self.gt_class_name_to_label_map = {p.name: i for i, p in enumerate(gt_class_paths)}

        for p in gt_class_paths:
            for img_path in p.iterdir():
                self.img_paths.append(img_path)
                self.gt_paths.append(
                    gt_path / p.name / f"{img_path.stem}_mask{img_path.suffix}"
                )
        self.transforms = transforms

    def __getitem__(self, index: int):
        img_path = self.img_paths[index]
        gt_path = self.gt_paths[index]

        label = self.gt_class_name_to_label_map[gt_path.parent.name]

        img = Image.open(img_path)
        img = img.convert("RGB")

        if self.transforms:
            img = np.array(img)
            img = self.transforms(image=img)["image"]

        return img, label

    def __len__(self) -> int:
        return len(self.img_paths)

In [ ]:
val_ds = ValidationDataset(val_path, gt_path, transforms=default_transforms)

good_label = val_ds.gt_class_name_to_label_map['good']
label_to_name_map = {v: k for k, v in val_ds.gt_class_name_to_label_map.items()}

val_dl = DataLoader(
    val_ds,
    batch_size=Config.batch_size,
    shuffle=False,
    num_workers=Config.num_workers,
)

In [ ]:
val_features = np.empty((len(val_ds), Config.n_feats), dtype=np.float32)
val_labels = np.zeros((len(val_ds)), dtype=np.uint32)

feature_extractor = feature_extractor.to(Config.device)
i_mem = 0

for i, (imgs, labels) in enumerate(val_dl):
    feats = get_features(imgs, feature_extractor, Config)
    n_samples = feats.shape[0]
    labels = get_ground_truths(labels, Config)

    val_features[i_mem : i_mem + n_samples] = feats
    val_labels[i_mem : i_mem + n_samples] = labels

    i_mem += n_samples

print("Validation features shape:", val_features.shape)
print("Validation labels shape:", val_labels.shape)

In [ ]:
ano_gt = (val_labels != good_label).astype(np.int32)

## PCA

Look at [Gaussian-AD code](https://github.com/ORippler/gaussian-ad-mvtec/blob/bc10bd736d85b750410e6b0e7ac843061e09511e/src/gaussian/model.py#L207) for PCA keeping features with least variance

In [ ]:
X_train = train_features
pca = PCA(n_components=None).fit(X_train)

In [ ]:
X_val = val_features
y = ano_gt
y_label = val_labels

In [ ]:
variance_thresholds = [0.9, 0.99]
variances = pca.explained_variance_ratio_.cumsum()

i_comp_thresholds = []
for variance_threshold in variance_thresholds:
    i_comp_thresholds.append((variances > variance_threshold).argmax())

# Normal PCA
pca_comps = pca.components_[: i_comp_thresholds[0]]
X_pca = np.matmul(X_val, pca_comps.T)

# Negative PCA
npca_comps = pca.components_[i_comp_thresholds[1] :]
X_npca = np.matmul(X_val, npca_comps.T)

print(X_pca.shape)
print(X_npca.shape)

In [ ]:
# n_dim = 2

# umap_for_all = umap.UMAP(n_components=n_dim)
# X_all_embed = umap_for_all.fit_transform(X_val)

# umap_for_pca = umap.UMAP(n_components=n_dim)
# X_pca_embed = umap_for_pca.fit_transform(X_pca)

# umap_for_npca = umap.UMAP(n_components=n_dim)
# X_npca_embed = umap_for_npca.fit_transform(X_npca)

In [ ]:
# p_all = plot_feature_samples(
#     X_all_embed, y, title="Feature embedding for all features", width=400, height=400, alpha=1.0
# )

# p_pca = plot_feature_samples(
#     X_pca_embed, y, title="Feature embedding after standard PCA", width=400, height=400
# )
# p_npca = plot_feature_samples(
#     X_npca_embed, y, title="Feature embedding after negative PCA", width=400, height=400
# )
# p = bokeh.layouts.row(p_all, p_pca, p_npca)
# show(p)

In [ ]:
# p_all = plot_labelled_feature_samples(
#     X_all_embed, y_label, label_to_name_map=label_to_name_map, title="Feature embedding for all features", width=400, height=400
# )

# p_pca = plot_labelled_feature_samples(
#     X_pca_embed, y_label, label_to_name_map=label_to_name_map, title="Feature embedding after standard PCA", width=400, height=400
# )
# p_npca = plot_labelled_feature_samples(
#     X_npca_embed, y_label, label_to_name_map=label_to_name_map, title="Feature embedding after negative PCA", width=400, height=400
# )
# p = bokeh.layouts.row(p_all, p_pca, p_npca)
# show(p)

In [ ]:
# p_npca = plot_labelled_feature_samples(
#     X_npca_embed,
#     y_label,
#     label_to_name_map=label_to_name_map,
#     title="Feature embedding after negative PCA",
#     width=800,
#     height=800,
#     alpha=1.0,
# )
# show(p_npca)

In [ ]:
n_dim = 3

umap_for_all = umap.UMAP(n_components=n_dim)
X_all_embed = umap_for_all.fit_transform(X_val)

umap_for_pca = umap.UMAP(n_components=n_dim)
X_pca_embed = umap_for_pca.fit_transform(X_pca)

umap_for_npca = umap.UMAP(n_components=n_dim)
X_npca_embed = umap_for_npca.fit_transform(X_npca)

In [ ]:
ax = plot_labelled_feature_3d_samples(X_all_embed, y_label, label_to_name_map=label_to_name_map, title="Feature embedding for all features")
plt.show()

In [ ]:
ax = plot_labelled_feature_3d_samples(X_pca_embed, y_label, label_to_name_map=label_to_name_map, title="Feature embedding after standard PCA")
plt.show()

In [ ]:
ax = plot_labelled_feature_3d_samples(X_npca_embed, y_label, label_to_name_map=label_to_name_map, title="Feature embedding after negative PCA")
plt.show()

## Anomaly Detection

In [ ]:
def mahalanobis_distance(
    values: np.ndarray, mean: np.ndarray, inv_covariance: np.ndarray
) -> np.ndarray:
    """Compute the batched mahalanobis distance.
    values is a batch of feature vectors.
    mean is either the mean of the distribution to compare, or a second
    batch of feature vectors.
    inv_covariance is the inverse covariance of the target distribution.
    """
    assert values.ndim == 2
    assert 1 <= mean.ndim <= 2
    assert len(inv_covariance.shape) == 2
    assert values.shape[1] == mean.shape[-1]
    assert mean.shape[-1] == inv_covariance.shape[0]
    assert inv_covariance.shape[0] == inv_covariance.shape[1]

    if mean.ndim == 1:  # Distribution mean.
        mean = np.expand_dims(mean, 0)
    x_mu = values - mean  # batch x features
    # Same as dist = x_mu.t() * inv_covariance * x_mu batch wise
    dist = np.einsum("im,mn,in->i", x_mu, inv_covariance, x_mu)
    return np.sqrt(dist)

In [ ]:
variance_thresholds = [0.99]
variances = pca.explained_variance_ratio_.cumsum()

i_comp_thresholds = []
for variance_threshold in variance_thresholds:
    i_comp_thresholds.append((variances > variance_threshold).argmax())

# Normal PCA
pca_comps = pca.components_[: i_comp_thresholds[0]]

train_features_pca = np.matmul(train_features, pca_comps.T)
val_features_pca = np.matmul(val_features, pca_comps.T)

print("PCA training features shape", train_features_pca.shape)
print("PCA validation features shape", val_features_pca.shape)

# Negative PCA
npca_comps = pca.components_[i_comp_thresholds[0]:]

train_features_npca = np.matmul(train_features, npca_comps.T)
val_features_npca = np.matmul(val_features, npca_comps.T)

print("NPCA training features shape", train_features_npca.shape)
print("NPCA validation features shape", val_features_npca.shape)

In [ ]:
# clf = LUNAR(n_neighbours=5)
# clf = IForest()
# clf = KNN(n_neighbors=5)
# clf.fit(train_features)

# model_path = output_path / 'clf.pkl'
# pickle.dump(clf, open(model_path, 'wb'))
# clf = pickle.load(open(model_path, 'rb'))

In [ ]:
mean_npca = np.mean(train_features_npca, axis=0)
lw_cov_npca = LedoitWolf().fit(train_features_npca)
inv_cov_npca = lw_cov_npca.precision_ 

ano_scores_npca = mahalanobis_distance(val_features_npca, mean_npca, inv_cov_npca)

fpr_img, tpr_img, thresholds_img = roc_curve(ano_gt, ano_scores_npca)
auroc_img = auc(fpr_img, tpr_img)

print(f"NPCA reduction, image-wise AUROC: {auroc_img:.5f}")

In [ ]:
mean_pca = np.mean(train_features_pca, axis=0)
lw_cov_pca = LedoitWolf().fit(train_features_pca)
inv_cov_pca = lw_cov_pca.precision_ 

ano_scores_pca = mahalanobis_distance(val_features_pca, mean_pca, inv_cov_pca)

fpr_img, tpr_img, thresholds_img = roc_curve(ano_gt, ano_scores_pca)
auroc_img = auc(fpr_img, tpr_img)

print(f"PCA reduction, image-wise AUROC: {auroc_img:.5f}")

In [ ]:
# plot_path = output_path / "ROC_curve.html"
# save_plot_from_notbook_for_jekyll(p, plot_path)

## Conclusion